In [1]:
import sys
import os

parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.append(parent_dir)

from chroma_functions import *
from utils import logger

/home/anjo/.conda/envs/BAp11/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chroma = ChromaDB("hdm_collection", "../chroma_storage")

2025-02-14 16:20:29 [INFO] chromadb.telemetry.product.posthog - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-14 16:20:29 [INFO] sentence_transformers.SentenceTransformer - Use pytorch device_name: cpu
2025-02-14 16:20:29 [INFO] sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: sentence-transformers/distiluse-base-multilingual-cased-v1
2025-02-14 16:20:46 [INFO] chroma_functions - ChromaDB instance initialized.


In [3]:
chroma.client.list_collections()

['hdm_collection', 'hdm_website']

In [5]:
entries = chroma.collection.get(include=["metadatas"])
len(entries["ids"])

539

In [ ]:
import chromadb

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="../chroma_storage")  # Or chromadb.Client() for in-memory DB

# Get your collection
collection = client.get_collection(name="hdm_collection")

# Check the distance function
print("Distance function:", collection.metadata.get("hnsw:space", "cosine")) 

Distance function: None


In [41]:
result = chroma.get_closest_document("test sentence")

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.43it/s]

2025-02-13 19:24:43 [WARNING] chromadb.segment.impl.vector.local_persistent_hnsw - Number of requested results 1 is greater than number of elements in index 0, updating n_results = 0


IndexError: list index out of range

In [ ]:
result[0]["metadata"]["token_count"]

620

In [ ]:
results = chroma.collection.get(
    where={"token_count": {"$gt": 5000}},  # Filtering condition
    include=["documents", "metadatas"]  # Include documents and metadata in the results
)

In [ ]:
len(results["ids"])

178

In [38]:
entries["ids"]

['webpage-a08f78c32e264befc8739074d26f16e762c51802',
 'webpage-4a833a4587c2d1efd178e5007be1b5757e47b006',
 'webpage-f85e5735919f88c5b1ece5afbdf06bb252d1c935',
 'webpage-b7093498716ee2127f6a8d04e5b5de6c5fc3d1e2',
 'webpage-481fcddd109c00423ddc3bd0f4802b399e2b82fa',
 'webpage-a4d98b0934ad0465948c490a9e071ea456b5a572',
 'webpage-5763830658321b78df9277110a276fb5a125fc18',
 'webpage-b3838a77236099b0e6bfe9c1fceb640ad517f8c4',
 'webpage-8a50c8c91e6edc7fb9060c26747e3bb152153527',
 'webpage-26adaf30646585276ee41b6a09fcb21095d4a79c',
 'webpage-65af2cdd6d227a109f2621009304bbaf278df4b6',
 'webpage-8aa955ead67200f997e84918119f392d04cdf191',
 'webpage-50cd7bbf78c628c58dfc868856ed0785df1f9ffc',
 'webpage-7f1fba113c4c1cfca40dc4212ca1f5b1b0deb012',
 'webpage-738d83cd34a0b97e08809682d8953cbe9516875c',
 'webpage-5fef242da6cbf6483cb9ee8198d29e8e5d522912',
 'webpage-8b589c17251d52f3e88ec3349c99f1aff92ef554',
 'webpage-17f1cec50e29b019d4e6e0d0c0f0cbf2619e8eb4',
 'webpage-4f6145b0925f750839cb5aeb42e781319dd5

In [39]:
chroma.collection.delete(
    ids=entries["ids"]
)

2025-02-13 19:23:51 [ERROR] chromadb.db.mixins.embeddings_queue - Exception occurred invoking consumer for subscription cfe19da0585c4f749cb798e40c83e815to topic persistent://default/default/09227d7a-72cf-419c-a500-7a22d82eaff9 database disk image is malformed


In [10]:
import re
pattern = r".*curdt.*"
# Fetch all documents (use a limit if needed for performance)
results = chroma.collection.get(include=["metadatas"])
print(results.keys())
print(results["ids"][0])
# Compile regex pattern
regex = re.compile(pattern)

# Filter in Python using regex
documents = [
    doc_id for doc_id, meta in zip(results.get("ids", []), results.get("metadatas", []))
    if "url" in meta and regex.search(meta["url"])
]

log.info(f"Found {len(documents)} matching documents for regex: {pattern}")
# results.get("ids")

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])
webpage-a08f78c32e264befc8739074d26f16e762c51802
2025-02-12 21:36:37 [INFO] chroma_functions - Found 785 matching documents for regex: .*curdt.*


In [16]:
len(documents)

785

In [17]:
chroma.collection.delete(
    ids=documents
)